# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 25 2023 8:48AM,261725,10,PRF-59245,Bio-PRF,Released
1,Apr 25 2023 8:48AM,261725,10,PRF-59249,Bio-PRF,Released
2,Apr 25 2023 8:48AM,261725,10,PRF-59255,Bio-PRF,Released
3,Apr 25 2023 8:48AM,261725,10,PRF-59258,Bio-PRF,Released
4,Apr 25 2023 8:48AM,261725,10,PRF-59261,Bio-PRF,Released
5,Apr 25 2023 8:48AM,261725,10,PRF-59264,Bio-PRF,Released
6,Apr 25 2023 8:48AM,261725,10,PRF-59268,Bio-PRF,Released
7,Apr 25 2023 8:48AM,261725,10,PRF-59271,Bio-PRF,Released
8,Apr 25 2023 8:48AM,261725,10,PRF-59274,Bio-PRF,Released
9,Apr 25 2023 8:48AM,261725,10,PRF-59277,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,261721,Released,8
14,261722,Released,1
15,261723,Released,5
16,261724,Released,2
17,261725,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261721,NaN,1.0,8.0
261722,NaN,NaN,1.0
261723,NaN,NaN,5.0
261724,NaN,NaN,2.0
261725,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261621,0.0,0.0,1.0
261622,0.0,0.0,50.0
261641,0.0,1.0,0.0
261649,0.0,0.0,1.0
261652,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261621,0,0,1
261622,0,0,50
261641,0,1,0
261649,0,0,1
261652,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261621,0,0,1
1,261622,0,0,50
2,261641,0,1,0
3,261649,0,0,1
4,261652,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261621,,,1
1,261622,,,50
2,261641,,1,
3,261649,,,1
4,261652,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 25 2023 8:48AM,261725,10,Bio-PRF
12,Apr 25 2023 8:46AM,261724,19,Else Nutrition
14,Apr 25 2023 8:46AM,261723,10,"Methapharm, Inc."
19,Apr 25 2023 8:40AM,261722,19,"Graystone, LLC"
20,Apr 25 2023 7:59AM,261721,19,"AdvaGen Pharma, Ltd"
29,Apr 24 2023 3:51PM,261701,19,"AdvaGen Pharma, Ltd"
46,Apr 24 2023 3:13PM,261692,19,"NAPP Technologies, LLC"
47,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc."
69,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd"
87,Apr 24 2023 12:15PM,261652,20,VITABIOTICS USA INC.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 25 2023 8:48AM,261725,10,Bio-PRF,,,12
1,Apr 25 2023 8:46AM,261724,19,Else Nutrition,,,2
2,Apr 25 2023 8:46AM,261723,10,"Methapharm, Inc.",,,5
3,Apr 25 2023 8:40AM,261722,19,"Graystone, LLC",,,1
4,Apr 25 2023 7:59AM,261721,19,"AdvaGen Pharma, Ltd",,1,8
5,Apr 24 2023 3:51PM,261701,19,"AdvaGen Pharma, Ltd",,1,16
6,Apr 24 2023 3:13PM,261692,19,"NAPP Technologies, LLC",,,1
7,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc.",,12,10
8,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd",11,7,
9,Apr 24 2023 12:15PM,261652,20,VITABIOTICS USA INC.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 25 2023 8:48AM,261725,10,Bio-PRF,12,,
1,Apr 25 2023 8:46AM,261724,19,Else Nutrition,2,,
2,Apr 25 2023 8:46AM,261723,10,"Methapharm, Inc.",5,,
3,Apr 25 2023 8:40AM,261722,19,"Graystone, LLC",1,,
4,Apr 25 2023 7:59AM,261721,19,"AdvaGen Pharma, Ltd",8,1,
5,Apr 24 2023 3:51PM,261701,19,"AdvaGen Pharma, Ltd",16,1,
6,Apr 24 2023 3:13PM,261692,19,"NAPP Technologies, LLC",1,,
7,Apr 24 2023 2:41PM,261683,20,"Exact-Rx, Inc.",10,12,
8,Apr 24 2023 1:31PM,261676,19,"AdvaGen Pharma, Ltd",,7,11
9,Apr 24 2023 12:15PM,261652,20,VITABIOTICS USA INC.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 25 2023 8:48AM,261725,10,Bio-PRF,12,,
1,Apr 25 2023 8:46AM,261724,19,Else Nutrition,2,,
2,Apr 25 2023 8:46AM,261723,10,"Methapharm, Inc.",5,,
3,Apr 25 2023 8:40AM,261722,19,"Graystone, LLC",1,,
4,Apr 25 2023 7:59AM,261721,19,"AdvaGen Pharma, Ltd",8,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 25 2023 8:48AM,261725,10,Bio-PRF,12.0,NaN,NaN
1,Apr 25 2023 8:46AM,261724,19,Else Nutrition,2.0,NaN,NaN
2,Apr 25 2023 8:46AM,261723,10,"Methapharm, Inc.",5.0,NaN,NaN
3,Apr 25 2023 8:40AM,261722,19,"Graystone, LLC",1.0,NaN,NaN
4,Apr 25 2023 7:59AM,261721,19,"AdvaGen Pharma, Ltd",8.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 25 2023 8:48AM,261725,10,Bio-PRF,12.0,0.0,0.0
1,Apr 25 2023 8:46AM,261724,19,Else Nutrition,2.0,0.0,0.0
2,Apr 25 2023 8:46AM,261723,10,"Methapharm, Inc.",5.0,0.0,0.0
3,Apr 25 2023 8:40AM,261722,19,"Graystone, LLC",1.0,0.0,0.0
4,Apr 25 2023 7:59AM,261721,19,"AdvaGen Pharma, Ltd",8.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,785089,17.0,1.0,0.0
19,1570236,28.0,9.0,11.0
20,1308227,63.0,12.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,785089,17.0,1.0,0.0
1,19,1570236,28.0,9.0,11.0
2,20,1308227,63.0,12.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,17.0,1.0,0.0
1,19,28.0,9.0,11.0
2,20,63.0,12.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,17.0
1,19,Released,28.0
2,20,Released,63.0
3,10,Executing,1.0
4,19,Executing,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20
Status,,,
Completed,0.0,11.0,0.0
Executing,1.0,9.0,12.0
Released,17.0,28.0,63.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20
0,Completed,0.0,11.0,0.0
1,Executing,1.0,9.0,12.0
2,Released,17.0,28.0,63.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20
0,Completed,0.0,11.0,0.0
1,Executing,1.0,9.0,12.0
2,Released,17.0,28.0,63.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()